# MCP Agent

- Requires distributions/masterclass-agents/run.yaml

In [13]:
from llama_stack_client import LlamaStackClient, Agent, AgentEventLogger


In [14]:
client = LlamaStackClient(
    base_url="http://localhost:8321",
)

In [15]:
client.models.list()

[Model(identifier='all-MiniLM-L6-v2', metadata={'embedding_dimension': 384.0}, api_model_type='embedding', provider_id='ollama', provider_resource_id='all-minilm:latest', type='model', model_type='embedding'),
 Model(identifier='meta-llama/Llama-3.2-3B-Instruct', metadata={}, api_model_type='llm', provider_id='ollama', provider_resource_id='llama3.2:3b-instruct-fp16', type='model', model_type='llm')]

In [16]:
client.tools.list()

[Tool(description='Insert documents into memory', identifier='insert_into_memory', parameters=[], provider_id='rag-runtime', provider_resource_id='insert_into_memory', tool_host='distribution', toolgroup_id='builtin::rag', type='tool', metadata=None),
 Tool(description='Search for information in a database.', identifier='knowledge_search', parameters=[Parameter(description='The query to search for. Can be a natural language sentence or keywords.', name='query', parameter_type='string', required=True, default=None)], provider_id='rag-runtime', provider_resource_id='knowledge_search', tool_host='distribution', toolgroup_id='builtin::rag', type='tool', metadata=None),
 Tool(description='Search the web for information', identifier='web_search', parameters=[Parameter(description='The query to search for', name='query', parameter_type='string', required=True, default=None)], provider_id='tavily-search', provider_resource_id='web_search', tool_host='distribution', toolgroup_id='builtin::webse

In [17]:
client.toolgroups.list()

[ToolGroup(identifier='builtin::rag', provider_id='rag-runtime', provider_resource_id='builtin::rag', type='tool_group', args=None, mcp_endpoint=None),
 ToolGroup(identifier='builtin::websearch', provider_id='tavily-search', provider_resource_id='builtin::websearch', type='tool_group', args=None, mcp_endpoint=None)]

In [18]:
# alternatively can have this in the run.yaml file
client.toolgroups.register(
    toolgroup_id="mcp::filesystem",
    provider_id="model-context-protocol",
    mcp_endpoint={"uri": "http://localhost:8000/sse"},
)

In [19]:
client.toolgroups.list()

[ToolGroup(identifier='builtin::rag', provider_id='rag-runtime', provider_resource_id='builtin::rag', type='tool_group', args=None, mcp_endpoint=None),
 ToolGroup(identifier='builtin::websearch', provider_id='tavily-search', provider_resource_id='builtin::websearch', type='tool_group', args=None, mcp_endpoint=None),
 ToolGroup(identifier='mcp::filesystem', provider_id='model-context-protocol', provider_resource_id='mcp::filesystem', type='tool_group', args=None, mcp_endpoint=McpEndpoint(uri='http://localhost:8000/sse'))]

In [20]:
from llama_stack_client.types.shared_params.agent_config import ToolConfig

agent = Agent(
    client,
    model="meta-llama/Llama-3.2-3B-Instruct",
    instructions="You are a helpful assistant.",
    tools=["mcp::filesystem"],
)

In [22]:
user_prompts = [
    "Hello",
    "do you have access to any tools?",
    "list the directories that you have access to",
]

session_id = agent.create_session("test-session")
for prompt in user_prompts:
    print(f"User> {prompt}")
    response = agent.create_turn(
        messages=[{"role": "user", "content": prompt}],
        session_id=session_id,
    )

    for log in AgentEventLogger().log(response):
        log.print()

User> Hello
inference> 

Hello! How can I assist you today?
User> do you have access to any tools?
inference> I do have access to a set of tools that can be invoked to perform various tasks related to file system operations, such as reading and writing files, creating directories, listing directory contents, and more. The list of available tools is provided in the format below:
User> list the directories that you have access to
inference> [list_allowed_directories()]
tool_execution> Tool:list_allowed_directories Args:{}
tool_execution> Tool:list_allowed_directories Response:{"type":"text","text":"Allowed directories:\n/Users/diego/src/ai-masterclass-2025/my-local-files","annotations":null}
inference> The tool `list_allowed_directories()` was invoked successfully. The output indicates that the user has access to a directory named "my-local-files" located at "/Users/diego/src/ai-masterclass-2025".
